## 2.1

In [3]:
import cv2

# Load the images
image1 = cv2.imread('C:/Users/adity/Downloads/CV_A3/panaroma_generation/1.jpg')
image2 = cv2.imread('C:/Users/adity/Downloads/CV_A3/panaroma_generation/2.jpg')

# Initialize the SIFT feature detector and extractor
sift = cv2.SIFT_create()

# Detect keypoints and compute descriptors for both images
keypoints1, descriptors1 = sift.detectAndCompute(image1, None)
keypoints2, descriptors2 = sift.detectAndCompute(image2, None)

# Draw keypoints on the images
image1_keypoints = cv2.drawKeypoints(image1, keypoints1, None)
image2_keypoints = cv2.drawKeypoints(image2, keypoints2, None)

# Display the images with keypoints
cv2.imshow('Image 1 with Keypoints', image1_keypoints)
cv2.imshow('Image 2 with Keypoints', image2_keypoints)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 2.2

In [4]:
import cv2

# Load the images
image1 = cv2.imread('C:/Users/adity/Downloads/CV_A3/panaroma_generation/1.jpg', cv2.IMREAD_GRAYSCALE)
image2 = cv2.imread('C:/Users/adity/Downloads/CV_A3/panaroma_generation/2.jpg', cv2.IMREAD_GRAYSCALE)

# Initialize the feature detector and extractor (e.g., SIFT)
sift = cv2.SIFT_create()

# Detect keypoints and compute descriptors for both images
keypoints1, descriptors1 = sift.detectAndCompute(image1, None)
keypoints2, descriptors2 = sift.detectAndCompute(image2, None)

# Initialize the feature matcher using brute-force matching
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)

# Match the descriptors using brute-force matching
matches_bf = bf.match(descriptors1, descriptors2)

# Sort the matches by distance (lower is better)
matches_bf = sorted(matches_bf, key=lambda x: x.distance)

# Draw the top N matches
num_matches = 50
image_matches_bf = cv2.drawMatches(image1, keypoints1, image2, keypoints2, matches_bf[:num_matches], None)

# Initialize the feature matcher using FLANN matching
index_params = dict(algorithm=0, trees=5)
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params, search_params)

# Match the descriptors using FLANN matching
matches_flann = flann.match(descriptors1, descriptors2)

# Sort the matches by distance (lower is better)
matches_flann = sorted(matches_flann, key=lambda x: x.distance)

# Draw the top N matches
image_matches_flann = cv2.drawMatches(image1, keypoints1, image2, keypoints2, matches_flann[:num_matches], None)

# Display the images with matches
cv2.imshow('Brute-Force Matching', image_matches_bf)
cv2.imshow('FLANN Matching', image_matches_flann)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 2.3

In [5]:
import cv2
import numpy as np

# Load the images
image1 = cv2.imread('C:/Users/adity/Downloads/CV_A3/panaroma_generation/1.jpg')
image2 = cv2.imread('C:/Users/adity/Downloads/CV_A3/panaroma_generation/2.jpg')

# Convert the images to grayscale
gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

# Initialize the feature detector and extractor (e.g., SIFT)
sift = cv2.SIFT_create()

# Detect keypoints and compute descriptors for both images
keypoints1, descriptors1 = sift.detectAndCompute(gray1, None)
keypoints2, descriptors2 = sift.detectAndCompute(gray2, None)

# Initialize the feature matcher using brute-force matching
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)

# Match the descriptors using brute-force matching
matches = bf.match(descriptors1, descriptors2)

# Extract the matched keypoints
src_points = np.float32([keypoints1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
dst_points = np.float32([keypoints2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

# Estimate the homography matrix using RANSAC
homography, mask = cv2.findHomography(src_points, dst_points, cv2.RANSAC, 5.0)

# Print the estimated homography matrix
print("Estimated Homography Matrix:")
print(homography)

Estimated Homography Matrix:
[[ 2.09497410e+02 -1.24818230e+01 -7.35850630e+04]
 [ 6.28724990e+01  1.27780663e+02 -3.85195947e+04]
 [ 2.15587391e-01 -4.63788961e-03  1.00000000e+00]]


## 2.4

In [9]:
# Warp the second image using the homography matrix
warped_image2 = cv2.warpPerspective(image2, homography, (image1.shape[1] + image2.shape[1], image1.shape[0]))

# Place the warped image and the first image side-by-side
result = np.zeros((max(image1.shape[0], warped_image2.shape[0]), image1.shape[1] + warped_image2.shape[1], 3), dtype=np.uint8)
result[:image1.shape[0], :image1.shape[1]] = image1
result[:warped_image2.shape[0], image1.shape[1]:] = warped_image2

# Display the side-by-side images
cv2.imshow('Warped Images Side-by-Side', result)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [14]:
import numpy as np
import imutils
import cv2


class Stitcher:
    def _init_(self):
        # determine if we are using OpenCV v3.X
        self.isv3 = imutils.is_cv3(or_better=True)

    def stitch(self, images, ratio=0.75, reprojThresh=4.0, showMatches=False):
    # unpack the images, then detect keypoints and extract
    # local invariant descriptors from them
        (imageB, imageA) = images
        (kpsA, featuresA) = self.detectAndDescribe(imageA)
        (kpsB, featuresB) = self.detectAndDescribe(imageB)
        # match features between the two images
        M = self.matchKeypoints(kpsA, kpsB, featuresA, featuresB, ratio, reprojThresh)
        M2 = self.matchKeypoints2(kpsA, kpsB, featuresA, featuresB, ratio, reprojThresh)
        # if the match is None, then there aren't enough matched
        # keypoints to create a panorama
        if M is None or M2 is None:
            return None

        # otherwise, apply a perspective warp to stitch the images together
        (matches, H, status) = M
        (matches1, H1, status1) = M2
        result = cv2.warpPerspective(
            imageA, H, (imageA.shape[1] + imageB.shape[1], imageA.shape[0])
        )
        result[0 : imageB.shape[0], 0 : imageB.shape[1]] = imageB

        # Add a separation between the two images
        separation_width = 10
        result_with_separation = np.zeros((result.shape[0], result.shape[1] + separation_width, 3), dtype=np.uint8)
        result_with_separation[:, :result.shape[1]] = result

        # Add separation
        result_with_separation[:, result.shape[1]:] = 255  # White separation line

        # check to see if the keypoint matches should be visualized
        if showMatches:
            vis = self.drawMatches(imageA, imageB, kpsA, kpsB, matches, status)
            vis1 = self.drawMatches(imageA, imageB, kpsA, kpsB, matches1, status1)
            # Combine vis and vis1 side by side with a separation between them
            # vis_combined = np.hstack((vis, np.full((vis.shape[0], separation_width, 3), 255), vis1))
            # return a tuple of the stitched image and the visualization
            return (result_with_separation, vis, vis1)

        # return the stitched image
        return result_with_separation


    def detectAndDescribe(self, image):
        # convert the image to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # check to see if we are using OpenCV 3.X
        # if self.isv3:
            # detect and extract features from the image
        descriptor = cv2.SIFT_create()
        (kps, features) = descriptor.detectAndCompute(image, None)
        # otherwise, we are using OpenCV 2.4.X
        # else:
        #     # detect keypoints in the image
        #     detector = cv2.FeatureDetector_create("SIFT")
        #     kps = detector.detect(gray)
        #     # extract features from the image
        #     extractor = cv2.DescriptorExtractor_create("SIFT")
        #     (kps, features) = extractor.compute(gray, kps)
        # convert the keypoints from KeyPoint objects to NumPy
        # arrays
        kps = np.float32([kp.pt for kp in kps])
        # return a tuple of keypoints and features
        return (kps, features)

    def matchKeypoints(self, kpsA, kpsB, featuresA, featuresB, ratio, reprojThresh):
        # compute the raw matches and initialize the list of actual
        # matches
        matcher = cv2.DescriptorMatcher_create("BruteForce")
        rawMatches = matcher.knnMatch(featuresA, featuresB, 2)
        matches = []
        # loop over the raw matches
        for m in rawMatches:
            # ensure the distance is within a certain ratio of each
            # other (i.e. Lowe's ratio test)
            if len(m) == 2 and m[0].distance < m[1].distance * ratio:
                matches.append((m[0].trainIdx, m[0].queryIdx))

        # computing a homography requires at least 4 matches
        if len(matches) > 4:
            # construct the two sets of points
            ptsA = np.float32([kpsA[i] for (_, i) in matches])
            ptsB = np.float32([kpsB[i] for (i, _) in matches])
            # compute the homography between the two sets of points
            (H, status) = cv2.findHomography(ptsA, ptsB, cv2.RANSAC, reprojThresh)
            # return the matches along with the homograpy matrix
            # and status of each matched point
            return (matches, H, status)
        # otherwise, no homograpy could be computed
        return None

    def matchKeypoints2(self, kpsA, kpsB, featuresA, featuresB, ratio, reprojThresh):
    # create the FLANN matcher object
        FLANN_INDEX_KDTREE = 0
        index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
        search_params = dict(checks=50)
        flann = cv2.FlannBasedMatcher(index_params, search_params)

        # compute the raw matches
        rawMatches = flann.knnMatch(featuresA, featuresB, k=2)
        matches = []

        # loop over the raw matches
        for m, n in rawMatches:
            # ensure the distance is within a certain ratio of each other (i.e. Lowe's ratio test)
            if m.distance < ratio * n.distance:
                matches.append((m.queryIdx, m.trainIdx))

        # set a threshold on the number of matches
        min_matches = 8
        if len(matches) >= min_matches:
            print("Number of matches before constructing ptsB:", len(matches))  # Debugging output
            
            # construct the two sets of points
            ptsA = np.float32([kpsA[i] for (_, i) in matches])
            
            # Filter out-of-bounds indices
            valid_indices = set(range(len(kpsB)))
            ptsB_indices = [i for (_, i) in matches if i in valid_indices]
            
            print("Indices for ptsB:", ptsB_indices)  # Debugging output
            
            # Construct ptsB using the filtered indices
            ptsB = np.float32([kpsB[i] for i in ptsB_indices])
            
            print("Shape of ptsA:", ptsA.shape)  # Debugging output
            print("Shape of ptsB:", ptsB.shape)  # Debugging output

            # compute the homography between the two sets of points
            (H, status) = cv2.findHomography(ptsA, ptsB, cv2.RANSAC, reprojThresh)

            # return the matches along with the homography matrix and status of each matched point
            return (matches, H, status)

        # otherwise, no homography could be computed
        return None




    def drawMatches(self, imageA, imageB, kpsA, kpsB, matches, status):
        # initialize the output visualization image
        (hA, wA) = imageA.shape[:2]
        (hB, wB) = imageB.shape[:2]
        vis = np.zeros((max(hA, hB), wA + wB, 3), dtype="uint8")
        vis[0:hA, 0:wA] = imageA
        vis[0:hB, wA:] = imageB
        # loop over the matches
        for (trainIdx, queryIdx), s in zip(matches, status):
            # only process the match if the keypoint was successfully
            # matched
            if s == 1:
                # draw the match
                ptA = (int(kpsA[queryIdx][0]), int(kpsA[queryIdx][1]))
                ptB = (int(kpsB[trainIdx][0]) + wA, int(kpsB[trainIdx][1]))
                cv2.line(vis, ptA, ptB, (0, 255, 0), 1)
        # return the visualization
        return vis


# load the two images and resize them to have a width of 400 pixels
# (for faster processing)
imageA = cv2.imread("C:/Users/adity/Downloads/CV_A3/panaroma_generation/1.jpg")
imageB = cv2.imread("C:/Users/adity/Downloads/CV_A3/panaroma_generation/2.jpg")
# stitch the images together to create a panorama
stitcher = Stitcher()
(result, vis) = stitcher.stitch([imageA, imageB], showMatches=True)
# show the images
cv2.imshow("Image A", imageA)
cv2.imshow("Image B", imageB)
cv2.imshow("Keypoint Matches Brute", vis)
cv2.imshow("Keypoint Matches FLANN", vis1)
cv2.imshow("Result", result)
cv2.waitKey(0)

Number of matches before constructing ptsB: 281
Indices for ptsB: [676, 671, 694, 691, 697, 698, 647, 749, 755, 408, 744, 756, 763, 978, 767, 771, 768, 780, 791, 807, 804, 812, 818, 827, 832, 833, 839, 842, 890, 859, 843, 895, 875, 853, 865, 1082, 886, 877, 902, 890, 86, 885, 86, 893, 896, 911, 1630, 915, 916, 923, 924, 1001, 921, 927, 988, 952, 1084, 1024, 949, 1082, 899, 1026, 975, 853, 890, 86, 853, 983, 1040, 977, 894, 978, 979, 906, 982, 990, 1005, 1013, 997, 1001, 988, 988, 1000, 1003, 1023, 78, 1017, 1024, 949, 1034, 1082, 1033, 980, 978, 1066, 1062, 1041, 1068, 952, 1056, 1084, 1138, 1094, 1105, 1106, 1035, 1089, 1040, 1109, 1132, 1112, 1099, 1113, 1000, 1120, 1137, 1118, 1205, 1136, 1131, 1031, 1143, 1194, 1093, 1133, 1133, 1152, 1144, 1161, 1146, 1165, 1181, 1180, 1184, 1225, 1188, 1189, 1177, 1190, 1193, 1196, 1200, 1201, 1250, 1203, 1191, 1192, 1210, 1198, 1199, 1202, 1215, 1231, 1223, 1244, 1249, 1318, 1306, 1288, 1317, 1318, 1281, 1291, 1337, 1314, 1323, 1312, 1327, 1401,

error: OpenCV(4.9.0) D:/a/opencv-python/opencv-python/opencv/modules/highgui/src/precomp.hpp:155: error: (-215:Assertion failed) src_depth != CV_16F && src_depth != CV_32S in function 'convertToShow'
